# Capítulo 02 - Instalação e Configuração

## 📋 Objetivo

Neste capítulo aprenderemos:
1. Instalar extensão Iceberg no DuckDB
2. Configurar ambientes (dev/prod)
3. Setup para cloud storage
4. Testes de conectividade
5. Troubleshooting comum

## 🔧 Requisitos

- DuckDB >= 1.4.0
- Python 3.8+
- PyIceberg (opcional)
- Acesso à internet (para extensões)

## 1. Instalação Básica

In [ ]:
import duckdb

print(f"DuckDB versão: {duckdb.__version__}")

# Verificar se >= 1.4.0
if duckdb.__version__ >= '1.4.0':
    print("✅ Versão compatível com Iceberg")
else:
    print("⚠️  Atualize: pip install --upgrade duckdb")

In [ ]:
# Função helper
def safe_install_ext(con, ext_name):
    try:
        con.execute(f"INSTALL {ext_name}")
        con.execute(f"LOAD {ext_name}")
        print(f"✅ Extension '{ext_name}' carregada")
        return True
    except Exception as e:
        print(f"❌ Erro: {e}")
        return False

print("Helper definida")

### 1.1 Instalar Extensão Iceberg

In [ ]:
con = duckdb.connect()

# Instalar e carregar
iceberg_ok = safe_install_ext(con, "iceberg")

if iceberg_ok:
    # Verificar
    result = con.execute("""
        SELECT extension_name, loaded, installed
        FROM duckdb_extensions()
        WHERE extension_name = 'iceberg'
    """).fetchone()
    
    print(f"\nIceberg: installed={result[2]}, loaded={result[1]}")

## 2. Setup Completo com Múltiplas Extensões

In [ ]:
def setup_duckdb_iceberg():
    """Configura DuckDB com extensões para Iceberg"""
    con = duckdb.connect()
    
    # Extensões necessárias
    extensions = ['iceberg', 'httpfs']  # httpfs para S3/HTTP
    
    for ext in extensions:
        print(f"Instalando {ext}...")
        con.execute(f"INSTALL {ext}")
        con.execute(f"LOAD {ext}")
    
    print("✅ Setup completo!")
    return con

# Usar
con = setup_duckdb_iceberg()

### 2.1 Setup para Cloud Providers

In [ ]:
con = duckdb.connect()

# Setup para S3, Azure, HTTP
safe_install_ext(con, "iceberg")
con.execute("INSTALL httpfs")
con.execute("INSTALL azure")

con.execute("LOAD httpfs")
con.execute("LOAD azure")

print("✅ Pronto para S3, Azure e HTTP!")

## 3. Configurações de Ambiente

### 3.1 Desenvolvimento Local

In [ ]:
import os

# Banco persistente para dev
con_dev = duckdb.connect('dev.duckdb')

# Extensões
safe_install_ext(con_dev, "iceberg")

# Configurações de desenvolvimento
con_dev.execute("SET threads = 2")
con_dev.execute("SET memory_limit = '2GB'")

# Criar diretório para testes
os.makedirs('iceberg_tables', exist_ok=True)

print("✅ Ambiente de desenvolvimento pronto!")
print(f"Threads: 2, Memory: 2GB")

### 3.2 Produção

In [ ]:
# Configuração para produção
con_prod = duckdb.connect('prod.duckdb')

# Extensões
safe_install_ext(con_prod, "iceberg")
con_prod.execute("INSTALL httpfs")
con_prod.execute("LOAD httpfs")

# Configurações de produção
con_prod.execute("SET threads = 8")
con_prod.execute("SET memory_limit = '16GB'")
# con_prod.execute("SET temp_directory = '/fast/ssd/temp'")

print("✅ Ambiente de produção pronto!")
print(f"Threads: 8, Memory: 16GB")
print("\n⚠️  NÃO use unsafe features em produção!")

## 4. Testes de Instalação

### 4.1 Teste Básico

In [ ]:
def test_iceberg_basic():
    """Testa instalação básica"""
    con = duckdb.connect()
    
    try:
        safe_install_ext(con, "iceberg")
        print("✅ Extensão carregada com sucesso")
        return True
    except Exception as e:
        print(f"❌ Erro: {e}")
        return False

test_iceberg_basic()

### 4.2 Teste Leitura Local

In [ ]:
def test_iceberg_local():
    """Testa leitura de tabela local"""
    con = duckdb.connect()
    safe_install_ext(con, "iceberg")
    
    # Usar tabela do capítulo anterior
    test_table = './iceberg_warehouse/default/my_table/metadata'
    
    if os.path.exists(test_table):
        files = [f for f in os.listdir(test_table) if f.endswith('.metadata.json')]
        if files:
            metadata_file = os.path.join(test_table, files[-1])
            try:
                result = con.execute(f"""
                    SELECT count(*) FROM iceberg_scan('{metadata_file}')
                """).fetchone()
                print(f"✅ Tabela lida: {result[0]} linhas")
                return True
            except Exception as e:
                print(f"❌ Erro: {e}")
                return False
        else:
            print("⚠️  Nenhum metadata.json encontrado")
            return None
    else:
        print(f"ℹ️  Tabela não encontrada em {test_table}")
        print("   Execute o Capítulo 01 primeiro")
        return None

test_iceberg_local()

## 5. Configuração de Conectividade

### 5.1 Verificar Conectividade S3

In [ ]:
import requests

def check_s3_connectivity():
    """Verifica conectividade com S3"""
    try:
        response = requests.get('https://s3.amazonaws.com', timeout=5)
        print(f"✅ S3 acessível (status: {response.status_code})")
        return True
    except Exception as e:
        print(f"❌ Problema de conectividade: {e}")
        print("   Verifique firewall/proxy")
        return False

check_s3_connectivity()

## 6. Classe de Configuração Reutilizável

In [ ]:
class IcebergConfig:
    """Configuração padrão para DuckDB + Iceberg"""
    
    @staticmethod
    def setup(db_path=':memory:', config=None):
        """
        Configura DuckDB com Iceberg
        
        Args:
            db_path: Caminho do banco (default: in-memory)
            config: Dict com configurações adicionais
        """
        # Configuração padrão
        default_config = {
            'threads': 4,
            'memory_limit': '4GB'
        }
        
        if config:
            default_config.update(config)
        
        # Conectar
        con = duckdb.connect(db_path, config=default_config)
        
        # Instalar extensões
        extensions = ['iceberg', 'httpfs']
        for ext in extensions:
            con.execute(f"INSTALL {ext}")
            con.execute(f"LOAD {ext}")
        
        return con

print("✅ Classe IcebergConfig definida")

### 6.1 Usar Classe

In [ ]:
# Desenvolvimento
dev_con = IcebergConfig.setup()
print("✅ Dev: memory, 4 threads, 4GB")

# Produção
prod_con = IcebergConfig.setup(
    db_path='prod_config.duckdb',
    config={'threads': 16, 'memory_limit': '32GB'}
)
print("✅ Prod: prod_config.duckdb, 16 threads, 32GB")

## 7. Troubleshooting

### 7.1 Problemas Comuns

In [ ]:
print("🔧 TROUBLESHOOTING GUIDE")
print("="*50)
print()
print("1️⃣  Extension not found")
print("   → Verifique internet")
print("   → DuckDB >= 1.4.0")
print()
print("2️⃣  Permission denied")
print("   → Verifique permissões de escrita")
print("   → Use :memory: para testar")
print()
print("3️⃣  Out of memory")
print("   → Reduza memory_limit")
print("   → Use temp_directory em SSD")
print()
print("4️⃣  S3 access denied")
print("   → Configure credenciais")
print("   → Verifique IAM permissions")
print()
print("5️⃣  Metadata not found")
print("   → Verifique path absoluto")
print("   → Confirme formato .metadata.json")

### 7.2 Teste de Diagnóstico

In [ ]:
print("🔍 DIAGNÓSTICO DO SISTEMA")
print("="*50)

# 1. Versão DuckDB
print(f"\n1. DuckDB: {duckdb.__version__}")
if duckdb.__version__ >= '1.4.0':
    print("   ✅ Compatível")
else:
    print("   ❌ Atualize!")

# 2. Extensões
print("\n2. Extensões:")
con_test = duckdb.connect()
for ext in ['iceberg', 'httpfs']:
    try:
        con_test.execute(f"INSTALL {ext}")
        con_test.execute(f"LOAD {ext}")
        print(f"   ✅ {ext}")
    except:
        print(f"   ❌ {ext}")

# 3. Conectividade
print("\n3. Conectividade:")
try:
    import requests
    r = requests.get('https://s3.amazonaws.com', timeout=3)
    print(f"   ✅ Internet OK")
except:
    print(f"   ⚠️  Internet/Proxy issue")

print("\n" + "="*50)
print("Diagnóstico completo!")

## ✅ Resumo

**Aprendemos:**
1. ✅ Instalar extensão Iceberg
2. ✅ Configurar ambientes dev/prod
3. ✅ Setup para cloud providers
4. ✅ Testes de instalação
5. ✅ Classe de configuração reutilizável
6. ✅ Troubleshooting

**Próximo:** Capítulo 03 - Leitura de Tabelas Iceberg